<a name="top"></a><img src="images/chisel_1024.png" alt="Chisel logo" style="width:480px;" />

# Module 2.6: More on ChiselTest
**Prev: [Putting it all Together: An FIR Filter](2.5_exercise.ipynb)**<br>
**Next: [Generators: Parameters](3.1_parameters.ipynb)**

## Motivation
Chisel 团队一直在致力于改进测试框架。"ChiselTest"，它提供了以下改进。

- 适用于单元测试和系统集成测试
- 设计用于可组合的抽象和分层
- 高度可用，通过使编写单元测试尽可能简单、无痛（避免样板代码和其他无意义的东西）和有用来鼓励单元测试

### Planned
- 能够针对多个后端和仿真器（可能需要链接到 Scala，如果测试向量不是静态的，或者在综合到 FPGA 时使用有限的测试构造 API 子集）
- 将包含在基本的 chisel3 中，以避免打包和依赖问题

## Setup

In [ ]:
val path = System.getProperty("user.dir") + "/source/load-ivy.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

In [ ]:
import chisel3._
import chisel3.util._
import chisel3.experimental._
import chisel3.experimental.BundleLiterals._
import chisel3.tester._
import chisel3.tester.RawTester.test

>本训练营需要与其他地方看到的 Chisel 导入有些细微的不同。`import chisel3.tester.RawTester.test` 引入了专门为训练营设计的 `test(...)` 版本。

---
# Basic Tester implementation

ChiselTest 从与 iotesters 相同的基本操作开始。以下是旧版 iotesters 和新版 ChiselTest 之间基本功能映射的简要总结

|        | iotesters             | ChiselTest            |
| :----  | :---                  | :---                |
| poke   | poke(c.io.in1, 6)     | c.io.in1.poke(6.U)    |
| peek   | peek(c.io.out1)       | c.io.out1.peek()      |
| expect | expect(c.io.out1, 6)  | c.io.out1.expect(6.U) |
| step   | step(1)               | c.io.clock.step(1)  |
| initiate | Driver.execute(...) { c => | test(...) { c => |


Let's start by looking at the simple pass through module from 2.1

In [ ]:
// Chisel Code, but pass in a parameter to set widths of ports
class PassthroughGenerator(width: Int) extends Module { 
  val io = IO(new Bundle {
    val in = Input(UInt(width.W))
    val out = Output(UInt(width.W))
  })
  io.out := io.in
}

Using the old style a simple test would look like this

```scala
val testResult = Driver(() => new Passthrough()) {
  c => new PeekPokeTester(c) {
    poke(c.io.in, 0)     // Set our input to value 0
    expect(c.io.out, 0)  // Assert that the output correctly has 0
    poke(c.io.in, 1)     // Set our input to value 1
    expect(c.io.out, 1)  // Assert that the output correctly has 1
    poke(c.io.in, 2)     // Set our input to value 2
    expect(c.io.out, 2)  // Assert that the output correctly has 2
  }
}
assert(testResult)   // Scala Code: if testResult == false, will throw an error
println("SUCCESS!!") // Scala Code: if we get here, our tests passed!
```



In [ ]:
test(new PassthroughGenerator(16)) { c =>
    c.io.in.poke(0.U)     // Set our input to value 0
    c.io.out.expect(0.U)  // Assert that the output correctly has 0
    c.io.in.poke(1.U)     // Set our input to value 1
    c.io.out.expect(1.U)  // Assert that the output correctly has 1
    c.io.in.poke(2.U)     // Set our input to value 2
    c.io.out.expect(2.U)  // Assert that the output correctly has 2
}

>为了说明 ChiselTest 是如何推进时钟的，我们可以在前面的示例中添加一些 `step` 操作。

In [ ]:
test(new PassthroughGenerator(16)) { c =>
    c.io.in.poke(0.U)     // Set our input to value 0
    c.clock.step(1)    // advance the clock
    c.io.out.expect(0.U)  // Assert that the output correctly has 0
    c.io.in.poke(1.U)     // Set our input to value 1
    c.clock.step(1)    // advance the clock
    c.io.out.expect(1.U)  // Assert that the output correctly has 1
    c.io.in.poke(2.U)     // Set our input to value 2
    c.clock.step(1)    // advance the clock
    c.io.out.expect(2.U)  // Assert that the output correctly has 2
}

---
## What to notice in the above example

ChiselTest 的 `test` 方法需要的样板代码更少。原来的 `PeekPokeTester` 现在已内置于流程中。

`poke` 和 `expect` 方法现在是每个 `io` 元素的一部分。这为测试器提供了重要的提示，以更好地检查类型。`peek` 和 `step` 操作现在也是 `io` 元素的方法。

另一个不同之处是，被 `poke` 和 `expect` 的值是 Chisel 字面量。尽管在这里非常简单，但在更高级和有趣的示例中，它也提供了更强的检查。这将在未来通过改进指定 `Bundle` 字面量的能力得到进一步增强。

## 具有 Decoupled 接口的模块

在本节中，我们将介绍一些 tester2 用于处理 `Decoupled` 接口的工具。`Decoupled` 采用一种 chisel 数据类型，并为其提供 `ready` 和 `valid` 信号。ChiselTest 提供了一些不错的工具，用于自动化和可靠地测试这些接口。

## 一个队列示例

`QueueModule` 传递的数据类型由 `ioType` 决定。`QueueModule` 内有 `entries` 状态元素，这意味着它可以在施加背压之前容纳那么多元素。

In [ ]:
class QueueModule[T <: Data](ioType: T, entries: Int) extends MultiIOModule {
  val in = IO(Flipped(Decoupled(ioType)))
  val out = IO(Decoupled(ioType))
  out <> Queue(in, entries)
}

## EnqueueNow and expectDequeueNow

*ChiselTest* 内置了一些方法，用于处理 IO 中带有 Decoupled 接口的电路。在这个示例中，我们将看到如何从 `queue` 中插入和提取值。

| 方法 | 描述 |
| :---   | :---        |
| enqueueNow | 向 `Decoupled` 输入接口添加（入队）一个元素 |
| expectDequeueNow | 从 `Decoupled` 输出接口移除（出队）一个元素 |

---

>注意：为了确保 `ready` 和 `valid` 字段在测试开始时都正确初始化，需要一些必要的样板代码，如 `initSource`，`setSourceClock` 等。

In [ ]:
test(new QueueModule(UInt(9.W), entries = 200)) { c =>
    // Example testsequence showing the use and behavior of Queue
    c.in.initSource()
    c.in.setSourceClock(c.clock)
    c.out.initSink()
    c.out.setSinkClock(c.clock)
    
    // 创建一个包含 200 个元素的测试向量，每个元素是 0 到 199 的 UInt
    val testVector = Seq.tabulate(200){ i => i.U }

    // 将 testVector 自身配对，然后对每一对元素进行入队和出队操作
    testVector.zip(testVector).foreach { case (in, out) =>
      c.in.enqueueNow(in)
      c.out.expectDequeueNow(out)
    }
}

## EnqueueSeq and DequeueSeq

现在我们将介绍两个用于单次操作的入队和出队操作的新方法。

| 方法 | 描述 |
| :---   | :---        |
| enqueueSeq | 继续从 `Seq` 向 `Decoupled` 输入接口逐个添加（入队）元素，直到序列耗尽 |
| expectDequeueSeq | 从 `Decoupled` 输出接口逐个移除（出队）元素，并将每个元素与 `Seq` 的下一个元素进行比较 |

---

> 注意：下面的示例运行良好，但如代码所示，`enqueueSeq` 必须在 `expectDequeueSeq` 开始之前完成。如果 `testVector` 的大小大于队列深度，则该示例将失败，因为队列会被填满，无法完成 `enqueueSeq`。试试看会出现什么样的失败情况。在下一节中，我们将展示如何正确构建这种类型的测试。

In [ ]:
test(new QueueModule(UInt(9.W), entries = 200)) { c =>
    // Example testsequence showing the use and behavior of Queue
    c.in.initSource()
    c.in.setSourceClock(c.clock)
    c.out.initSink()
    c.out.setSinkClock(c.clock)
    
    val testVector = Seq.tabulate(100){ i => i.U }

    c.in.enqueueSeq(testVector)
    c.out.expectDequeueSeq(testVector)
}

> 上一节中还有一个重要的收获是，我们刚刚看到的函数 `enqueueNow`、`enqueueSeq`、`expectDequeueNow` 和 `expectDequeueSeq` 并不是 ChiselTest 中复杂的特例逻辑。相反，它们是 ChiselTest 鼓励你从 ChiselTest 原语构建的测试框架的示例。要了解这些方法是如何实现的，请查看 [TestAdapters.scala](https://github.com/ucb-bar/chisel-testers2/blob/d199c5908828d0be5245f55fce8a872b2afb314e/src/main/scala/chisel3/tester/TestAdapters.scala)。

# Fork and Join in ChiselTest

在本节中，我们将介绍如何并行运行单元测试的部分内容。为此，我们将引入 testers2 的两个新特性。

| 方法 | 描述 |
| :---   | :---        |
| fork   | 启动一个并行代码块，可以通过在前一个 fork 代码块末尾添加 .fork 来并行运行其他 fork |
| join | 将多个相关的 fork 重新合并回调用线程 |

---

在下面的示例中，两个 `fork` 被链接在一起，然后使用 `join`。在第一个 `fork` 代码块中，`enqueueSeq` 将继续添加元素直到耗尽。在第二个 `fork` 代码块中，每个周期当数据可用时将执行 `expectDequeueSeq`。

>由 fork 创建的线程按照代码中指定的顺序以确定性的方式运行，并且某些依赖于其他线程的易出错操作在运行时检查时被禁止。

In [ ]:
test(new QueueModule(UInt(9.W), entries = 200)) { c =>
    // Example testsequence showing the use and behavior of Queue
    c.in.initSource()
    c.in.setSourceClock(c.clock)
    c.out.initSink()
    c.out.setSinkClock(c.clock)
    
    val testVector = Seq.tabulate(300){ i => i.U }

    fork {
        c.in.enqueueSeq(testVector)
    }.fork {
        c.out.expectDequeueSeq(testVector)
    }.join()
}

## Using Fork and Join with GCD

在本节中，我们将使用 `fork` 和 `join` 方法来实现 **GCD**（最大公约数）的测试。让我们从定义 IO bundles 开始。我们将在这里添加一些样板代码，以便我们能够使用 `Bundle` *字面量*。正如注释所说，希望我们很快就能支持字面量支持代码的自动生成。

In [ ]:
class GcdInputBundle(val w: Int) extends Bundle {
  val value1 = UInt(w.W)
  val value2 = UInt(w.W)
}

In [ ]:
class GcdOutputBundle(val w: Int) extends Bundle {
  val value1 = UInt(w.W)
  val value2 = UInt(w.W)
  val gcd    = UInt(w.W)
}

现在让我们来看一个 **GCD** 的 *Decoupled* 版本。我们在这里稍作修改，使用了 `Decoupled` 包装器，为输入和输出 Bundle 添加了 `ready` 和 `valid` 信号。`Flipped` 包装器将 `Decoupled` 的 `GcdInputBundle`（默认情况下创建为输出）转换为每个字段的相反方向（递归地）。`Decoupled` 的捆绑参数的数据元素放置在顶层字段 `bits` 中。

In [ ]:
/**
  * Compute GCD using subtraction method.
  * Subtracts the smaller of registers x and y from the larger until register y is zero.
  * value input register x is then the Gcd
  * returns a packet of information with the two input values and their GCD
  */
class DecoupledGcd(width: Int) extends MultiIOModule {

  val input = IO(Flipped(Decoupled(new GcdInputBundle(width))))
  val output = IO(Decoupled(new GcdOutputBundle(width)))

  val xInitial    = Reg(UInt())
  val yInitial    = Reg(UInt())
  val x           = Reg(UInt())
  val y           = Reg(UInt())
  val busy        = RegInit(false.B)
  val resultValid = RegInit(false.B)

  input.ready := ! busy
  output.valid := resultValid
  output.bits := DontCare

  when(busy)  {
    // during computation keep subtracting the smaller from the larger
    when(x > y) {
      x := x - y
    }.otherwise {
      y := y - x
    }
    when(y === 0.U) {
      // when y becomes zero computation is over,
      // signal valid data to output if the output is ready
      output.bits.value1 := xInitial
      output.bits.value2 := yInitial
      output.bits.gcd := x
      output.valid := true.B
      busy := ! output.ready
    }
  }.otherwise {
    when(input.valid) {
      // valid data available and no computation in progress, grab new values and start
      val bundle = input.deq()
      x := bundle.value1
      y := bundle.value2
      xInitial := bundle.value1
      yInitial := bundle.value2
      busy := true.B
      resultValid := false.B
    }
  }
}

我们的测试看起来与之前的队列测试非常相似。但是，由于计算需要多个周期，因此在每次计算 GCD 时，输入入队过程会被阻塞。好消息是，这种测试方法对于不同的 Decoupled 电路来说既简单又一致。

这里还引入了新的 Chisel3 `Bundle` 字面量表示法。考虑以下代码行：
```scala
new GcdInputBundle(16).Lit(_.value1 -> x.U, _.value2 -> y.U)
```
上面定义的 `GcdInputBundle` 有两个字段 `value1` 和 `value2`。我们通过先创建一个 bundle，然后调用其 `.Lit` 方法来创建一个 bundle 字面量。该方法接受一个键/值对的可变参数列表，其中键（例如 `_.value`）是字段名称，值（例如 x.U）是 Chisel 硬件字面量，即将 Scala `Int` x 转换为 Chisel `UInt` 字面量。字段名前的 `_.` 是必要的，以便将名称值绑定到 bundle 内部。

>这可能不是完美的表示法，但在广泛的开发讨论中，它被认为是在减少样板代码和 Scala 中可用的符号限制之间的最佳平衡。

In [ ]:
test(new DecoupledGcd(16)) { dut =>
  dut.input.initSource().setSourceClock(dut.clock)
  dut.output.initSink().setSinkClock(dut.clock)

  val testValues = for { x <- 1 to 10; y <- 1 to 10} yield (x, y)
  val inputSeq = testValues.map { case (x, y) =>
    (new GcdInputBundle(16)).Lit(_.value1 -> x.U, _.value2 -> y.U)
  }
  val resultSeq = testValues.map { case (x, y) =>
    new GcdOutputBundle(16).Lit(_.value1 -> x.U, _.value2 -> y.U, _.gcd -> BigInt(x).gcd(BigInt(y)).U)
  }

  fork {
    dut.input.enqueueSeq(inputSeq)
  }.fork {
    for (expected <- resultSeq) {
      dut.output.expectDequeue(expected)
      dut.clock.step(5) // wait some cycles before receiving the next output to create backpressure
    }
  }.join()
}


---
# You're done!

[Return to the top.](#top)